In [1]:
import os
import warnings


In [2]:
# os.environ["PYTHONWARNINGS"]="ignore"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"
warnings.filterwarnings("ignore")
# os.environ

In [3]:

import glob

import numpy as np
# from ray.tune.search.hyperopt import HyperOptSearch
import optuna
import pandas as pd
import torch
from IPython.display import display
from neuralforecast.auto import AutoLSTM, AutoNBEATS, AutoNHITS, AutoPatchTST
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.numpy import mae, mse
from neuralforecast.losses.pytorch import MSE
from ray import tune
from statsforecast import StatsForecast

from utils.utils import build_market_image

# optuna.logging.set_verbosity(optuna.logging.WARNING) # Use this to disable training prints from optuna

In [4]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH','BNB/ETH','XRP/ETH',"ADA/ETH",
            'SOL/BTC','SOL/BNB',
            'XRP/BTC','XRP/BNB',
            'ADA/BTC','ADA/BNB',
            ]
target_pair='EUR_USD'
time_frame="1h"

In [5]:
data=build_market_image(target_pair='EUR_USD',time_frame='1h',axis=0,dir='data_forex')

data=data.groupby('symbol').apply(lambda x: x[:pd.Timestamp('2024-01-01')])

data=data.reset_index(level=0,drop=True).reset_index()
id_columns=['symbol']


In [6]:
model=NeuralForecast.load("FX_forecasting_model/")

Seed set to 1
Seed set to 8
Seed set to 18
Seed set to 15


In [ ]:
sub_model=model.models[0]
context_length=max([sub_model.hparams['input_size'] for sub_model in model.models])
context_length

In [ ]:
front=['y','ds','symbol']
data=data[front+[c for c in data.columns if c not in front]]
data=data[front]

In [ ]:
eth_data=data[data['symbol']==target_pair].copy()
# data=eth_data
split=context_length

test=data.groupby('symbol').tail(split)
test.groupby('symbol').apply(lambda x: len(x))



In [ ]:
preds=model.predict(test)

In [ ]:
StatsForecast.plot(test,preds,id_col='symbol', engine='matplotlib')
